In [4]:
import pyrealsense2 as rs
import numpy as np
import cv2
import mediapipe as mp
import open3d as o3d

class RealSenseFaceScanner:
    def __init__(self):
        # Initialize MediaPipe Face Mesh with refined landmarks for iris detection
        self.mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.FACEMESH_TESSELATION = mp.solutions.face_mesh.FACEMESH_TESSELATION  # Added for connections

        # Initialize RealSense pipeline and configuration
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        self.config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

        # Alignment object to align depth to color
        self.align = rs.align(rs.stream.color)

        # Post-processing filters
        # Decimation filter (optional, commented out for now as it's low priority)
        # self.decimation = rs.decimation_filter()
        # self.decimation.set_option(rs.option.filter_magnitude, 2)

        # Spatial filter for edge-preserving smoothing
        self.spatial = rs.spatial_filter()
        self.spatial.set_option(rs.option.filter_magnitude, 5)
        self.spatial.set_option(rs.option.filter_smooth_alpha, 0.5)
        self.spatial.set_option(rs.option.filter_smooth_delta, 20)
        self.spatial.set_option(rs.option.holes_fill, 2)  # Small hole filling

        # Temporal filter for reducing temporal noise
        self.temporal = rs.temporal_filter()
        self.temporal.set_option(rs.option.filter_smooth_alpha, 0.4)
        self.temporal.set_option(rs.option.filter_smooth_delta, 20)
        self.temporal.set_option(rs.option.holes_fill, 3)  # Persistence for temporal

        # Hole filling filter (critical for handling occlusions like hair/glasses)
        self.hole_filling = rs.hole_filling_filter(2)  # 2: nearest neighbor filling

        # Key landmarks for measurements (MediaPipe indices)
        self.key_landmarks = {
            1: 'nose_tip',      # Nose tip for distance to camera
            10: 'forehead',     # Forehead for face height
            152: 'chin',        # Chin for face height
            129: 'left_nose_wing',  # Left nose wing for nose width
            358: 'right_nose_wing', # Right nose wing for nose width
            468: 'right_iris',  # Right iris center (note: MediaPipe labels right as 468, left as 473)
            473: 'left_iris',   # Left iris center
            33: 'left_eye_corner',  # Left eye outer corner (fallback for IPD)
            263: 'right_eye_corner' # Right eye outer corner (fallback for IPD)
        }

        # [BỔSUNG] Biến lưu tọa độ khi người dùng click chuột
        self.clicked_point = None

    # [BỔSUNG] Hàm callback để bắt sự kiện chuột từ OpenCV
    def mouse_callback(self, event, x, y, flags, param):
        """
        Mouse event callback: Captures left mouse clicks and stores pixel coordinates
        for interactive depth measurement.
        :param event: Event type (LBUTTONDOWN, etc.)
        :param x, y: Pixel coordinates of mouse click
        """
        if event == cv2.EVENT_LBUTTONDOWN:  # Khi nhấn chuột trái
            self.clicked_point = (x, y)     # Lưu tọa độ (u, v) lại
            print(f"[Mouse Event] Clicked at pixel: ({x}, {y})")

    def get_average_depth(self, depth_frame, x, y, height, width, window_size=3):
        """
        Get the average depth in a small window around (x, y) to handle invalid (0) depths.
        This helps with noisy or missing depth values, e.g., on hair or reflective surfaces.
        :param depth_frame: Aligned depth frame
        :param x, y: Pixel coordinates
        :param height, width: Frame dimensions
        :param window_size: Size of the square window (odd number)
        :return: Average depth (meters) or 0 if no valid depths found
        """
        depths = []
        half = window_size // 2
        for dy in range(-half, half + 1):
            for dx in range(-half, half + 1):
                nx, ny = x + dx, y + dy
                if 0 <= nx < width and 0 <= ny < height:
                    d = depth_frame.get_distance(nx, ny)
                    if d > 0:
                        depths.append(d)
        return np.mean(depths) if depths else 0.0

    def show_voxel_grid(self, depth_frame):
        """
        Generate and display a downsampled voxel grid from the depth frame's point cloud.
        Includes white point coloring, coordinate axes for orientation, and proper flip
        to handle RealSense vs Open3D coordinate system differences.
        :param depth_frame: Processed depth frame
        """
        print("[Snapshot] Processing 3D data...")
        
        # 1. Create Point Cloud from RealSense
        pc = rs.pointcloud()
        points = pc.calculate(depth_frame)
        
        # 2. Extract vertices
        vertices = np.asanyarray(points.get_vertices()).view(np.float32).reshape(-1, 3)
        
        # Check if point cloud has data
        if len(vertices) == 0:
            print("[Warning] Point cloud is empty! (Check depth camera)")
            return

        print(f"[Snapshot] Raw points: {len(vertices)}")

        # 3. Create Open3D Point Cloud
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(vertices)
        
        # --- CRITICAL FIX: Color points & flip coordinates ---
        
        # a. Paint all points WHITE to stand out on black background
        pcd.paint_uniform_color([1, 1, 1]) 
        
        # b. Downsample with smaller voxel size for better detail
        pcd_down = pcd.voxel_down_sample(voxel_size=0.03)  # 0.03m for finer grid
        print(f"[Snapshot] Voxel points: {len(pcd_down.points)}")

        # c. Create coordinate frame (X=Red, Y=Green, Z=Blue) for orientation reference
        # Size is 0.5 meters
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=[0, 0, 0])

        # d. Flip point cloud (RealSense and Open3D have opposite Y and Z axes)
        # Flip Y and Z so the face appears upright instead of upside-down
        pcd_down.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

        # 4. Display
        print("[Snapshot] Opening 3D Viewer... (Close window to continue)")
        o3d.visualization.draw_geometries([pcd_down, mesh_frame], 
                                          window_name="3D Voxel Grid Snapshot",
                                          width=800, height=600)

    def run(self):
        pipeline_started = False
        try:
            print("MediaPipe version:", mp.__version__)  # Debug version

            # Debugging: Check devices and USB type
            ctx = rs.context()
            devices = ctx.query_devices()
            if len(devices) == 0:
                raise RuntimeError("No RealSense devices detected. Check connection.")
            print(f"Detected {len(devices)} RealSense device(s).")
            dev = devices[0]
            usb_type = dev.get_info(rs.camera_info.usb_type_descriptor)
            print(f"Device: {dev.get_info(rs.camera_info.name)}, USB Type: {usb_type}")
            
            # Debugging: List supported stream profiles
            print("Supported stream profiles:")
            for sensor in dev.query_sensors():
                print(f"Sensor: {sensor.get_info(rs.camera_info.name)}")
                for profile in sensor.get_stream_profiles():
                    if profile.stream_type() == rs.stream.depth or profile.stream_type() == rs.stream.color:
                        print(profile)

            # Try starting with primary config
            try:
                self.pipeline.start(self.config)
                pipeline_started = True
                print("RealSense camera started successfully with primary config.")
            except RuntimeError as e:
                print(f"Primary config failed: {e}. Falling back to lower resolution.")
                # Fallback config (safer for USB2/compatibility)
                fallback_config = rs.config()
                fallback_config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
                fallback_config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
                self.pipeline.start(fallback_config)
                pipeline_started = True
                print("Started with fallback config (640x480).")

            # [BỔSUNG] Đăng ký cửa sổ & chuột (Trước vòng lặp while True)
            window_name = '3D Facial Analysis & Interactive Measure'
            cv2.namedWindow(window_name)
            cv2.setMouseCallback(window_name, self.mouse_callback)

            while True:
                # Wait for frames and align depth to color
                frames = self.pipeline.wait_for_frames()
                aligned_frames = self.align.process(frames)
                depth_frame = aligned_frames.get_depth_frame()
                color_frame = aligned_frames.get_color_frame()

                if not depth_frame or not color_frame:
                    continue

                # Apply post-processing filters in order
                # depth_frame = self.decimation.process(depth_frame)  # Optional
                depth_frame = self.spatial.process(depth_frame)
                depth_frame = self.temporal.process(depth_frame)
                depth_frame = self.hole_filling.process(depth_frame)
                
                # CRITICAL FIX: Cast back to depth_frame to get .get_distance() method
                depth_frame = depth_frame.as_depth_frame()

                # Get image data as NumPy arrays
                color_image = np.asanyarray(color_frame.get_data())
                print("Color frame shape:", color_image.shape)  # Debug
                depth_image = np.asanyarray(depth_frame.get_data())  # For reference if needed

                # Convert to RGB for MediaPipe processing
                rgb_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
                results = self.mp_face_mesh.process(rgb_image)
                if results.multi_face_landmarks:
                    print("Face detected!")  # Debug
                else:
                    print("No face detected.")  # Debug

                # Get frame dimensions
                height, width = color_image.shape[:2]

                # Draw face mesh and compute 3D points if face detected
                if results.multi_face_landmarks:
                    for face_landmarks in results.multi_face_landmarks:
                        # Vẽ lưới mặt với màu xanh lá đơn giản, không dùng style mặc định gây lỗi
                        self.mp_drawing.draw_landmarks(
                            image=color_image,
                            landmark_list=face_landmarks,
                            connections=self.FACEMESH_TESSELATION,
                            landmark_drawing_spec=None,  # Không vẽ chấm tròn
                            connection_drawing_spec=self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
                        )

                        # Get color intrinsics for deprojection
                        color_intrinsics = color_frame.profile.as_video_stream_profile().intrinsics

                        # Compute 3D points for key landmarks
                        points_3d = {}
                        for idx, name in self.key_landmarks.items():
                            lm = face_landmarks.landmark[idx]
                            u = int(lm.x * width)
                            v = int(lm.y * height)
                            # Get average depth in a small window to handle invalid values
                            depth = self.get_average_depth(depth_frame, u, v, height, width)
                            if depth > 0:
                                # Deproject pixel to 3D point
                                # Math: The deprojection transforms 2D pixel (u, v) with depth d to 3D (X, Y, Z) in meters
                                # using camera intrinsics (fx, fy, ppx, ppy): X = (u - ppx) * d / fx, Y = (v - ppy) * d / fy, Z = d
                                point_3d = rs.rs2_deproject_pixel_to_point(color_intrinsics, [u, v], depth)
                                points_3d[name] = np.array(point_3d)

                        # Calculate and display biometric measurements
                        text_pos = 30
                        if 'nose_tip' in points_3d:
                            # Distance to camera: Euclidean norm from (0,0,0) to nose tip
                            # Math: dist = sqrt(X^2 + Y^2 + Z^2)
                            dist_cam = np.linalg.norm(points_3d['nose_tip'])
                            color = (0, 255, 0) if dist_cam >= 0.4 else (0, 0, 255)
                            cv2.putText(color_image, f"Distance to Camera: {dist_cam:.2f} m", (10, text_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                            text_pos += 30

                        if 'left_iris' in points_3d and 'right_iris' in points_3d:
                            # IPD: Euclidean distance between iris centers
                            # Math: dist = sqrt( (X2 - X1)^2 + (Y2 - Y1)^2 + (Z2 - Z1)^2 )
                            ipd = np.linalg.norm(points_3d['left_iris'] - points_3d['right_iris'])
                            cv2.putText(color_image, f"IPD: {ipd:.2f} m", (10, text_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                            text_pos += 30
                        elif 'left_eye_corner' in points_3d and 'right_eye_corner' in points_3d:
                            # Fallback to eye corners if iris not available
                            ipd = np.linalg.norm(points_3d['left_eye_corner'] - points_3d['right_eye_corner'])
                            cv2.putText(color_image, f"IPD (fallback): {ipd:.2f} m", (10, text_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                            text_pos += 30

                        if 'forehead' in points_3d and 'chin' in points_3d:
                            face_height = np.linalg.norm(points_3d['forehead'] - points_3d['chin'])
                            cv2.putText(color_image, f"Face Height: {face_height:.2f} m", (10, text_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                            text_pos += 30

                        if 'left_nose_wing' in points_3d and 'right_nose_wing' in points_3d:
                            nose_width = np.linalg.norm(points_3d['left_nose_wing'] - points_3d['right_nose_wing'])
                            cv2.putText(color_image, f"Nose Width: {nose_width:.2f} m", (10, text_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                            text_pos += 30

                # [BỔSUNG] Logic hiển thị điểm click chuột
                if self.clicked_point:
                    cx, cy = self.clicked_point
                    
                    # 1. Lấy độ sâu tại điểm click (Dùng hàm get_average_depth có sẵn)
                    dist_click = self.get_average_depth(depth_frame, cx, cy, height, width)
                    
                    # 2. Vẽ dấu cộng màu đỏ để đánh dấu
                    cv2.drawMarker(color_image, (cx, cy), (0, 0, 255), cv2.MARKER_CROSS, 20, 2)
                    
                    # 3. Hiển thị text khoảng cách
                    if dist_click > 0:
                        text = f"Clicked Depth: {dist_click:.3f} m"
                        # Vẽ nền đen nhỏ cho chữ dễ đọc
                        cv2.rectangle(color_image, (cx + 10, cy - 25), (cx + 250, cy + 5), (0, 0, 0), -1)
                        cv2.putText(color_image, text, (cx + 10, cy - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
                        print(f"[Measurement] Clicked depth at ({cx}, {cy}): {dist_click:.3f} m")

                # Display the image (AFTER all drawing operations)
                cv2.imshow(window_name, color_image)

                # Handle key presses
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    # Snapshot and show voxel grid on 's' press
                    self.show_voxel_grid(depth_frame)
                elif key == ord('c'):
                    # Clear clicked point on 'c' press
                    self.clicked_point = None
                    print("[Control] Clicked point cleared")

        except RuntimeError as e:
            print(f"Error starting RealSense camera: {e}. Ensure the camera is connected via USB 3.0 and no other apps are using it.")
        except Exception as e:
            print(f"Unexpected error: {e}")
        finally:
            # Cleanup - only stop if pipeline was started
            if pipeline_started:
                self.pipeline.stop()
            cv2.destroyAllWindows()

# Usage: Instantiate and run the scanner

if __name__ == "__main__":
    scanner = RealSenseFaceScanner()
    scanner.run()

MediaPipe version: 0.10.9
Detected 1 RealSense device(s).
Error starting RealSense camera: MFCreateDeviceSource(_device_attrs, &_source) returned: HResult 0x800701b1: "A device which does not exist was specified.". Ensure the camera is connected via USB 3.0 and no other apps are using it.


In [22]:
import pyrealsense2 as rs
import numpy as np
import cv2
import mediapipe as mp
import open3d as o3d
import os

class RealSenseFaceScanner:
    def __init__(self):
        # 1. Khởi tạo MediaPipe Face Mesh
        self.mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.FACEMESH_TESSELATION = mp.solutions.face_mesh.FACEMESH_TESSELATION

        # 2. Khởi tạo RealSense
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        # Sử dụng cấu hình ổn định nhất cho D455
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        self.align = rs.align(rs.stream.color)

        # 3. Các bộ lọc xử lý nhiễu
        self.spatial = rs.spatial_filter()
        self.temporal = rs.temporal_filter()
        self.hole_filling = rs.hole_filling_filter(2) 

        self.clicked_point = None

    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.clicked_point = (x, y)
            print(f"[Sự kiện chuột] Tọa độ: ({x}, {y})")

    def get_average_depth(self, depth_frame, x, y, height, width, window_size=5):
        depths = []
        half = window_size // 2
        for dy in range(-half, half + 1):
            for dx in range(-half, half + 1):
                nx, ny = x + dx, y + dy
                if 0 <= nx < width and 0 <= ny < height:
                    d = depth_frame.get_distance(nx, ny)
                    if d > 0: depths.append(d)
        return np.mean(depths) if depths else 0.0

    def show_3d_snapshot(self, depth_frame, color_frame):
        """ 
        PHIÊN BẢN CÓ MÀU THẬT - LỌC NHIỄU TUYỆT ĐỐI:
        - Sử dụng Color Mapping để đắp màu thực tế lên mặt.
        - Lọc Statistical Outlier để xóa vệt dọc.
        - Tạo Mesh có màu sắc để Meshmixer không báo lỗi.
        """
        try:
            print("\n" + "="*50)
            print("[3D SNAPSHOT] ĐANG TẠO MÔ HÌNH 3D CÓ MÀU SẮC...")
            
            # 1. Lấy dữ liệu màu thực tế
            color_image = np.asanyarray(color_frame.get_data())
            # Chuẩn hóa màu sang RGB dải [0, 1] cho Open3D
            color_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

            # 2. Tạo Point Cloud và ánh xạ vân bề mặt
            pc = rs.pointcloud()
            pc.map_to(color_frame)
            points = pc.calculate(depth_frame)
            v = points.get_vertices()
            verts = np.asanyarray(v).view(np.float32).reshape(-1, 3)
            
            # 3. Lọc vùng mặt (Crop) - Giới hạn 0.2m đến 0.8m để tránh nhiễu xa
            valid_mask = (verts[:, 2] > 0.1) & (verts[:, 2] < 1.0) & np.all(np.isfinite(verts), axis=1)
            if np.sum(valid_mask) < 2000:
                print("[LỖI] Không đủ dữ liệu mặt. Hãy ngồi gần camera hơn!")
                return

            valid_verts = verts[valid_mask]
            # Lấy màu tương ứng cho từng điểm
            valid_colors = color_rgb.reshape(-1, 3)[valid_mask]

            # 4. Tạo PointCloud và xóa vệt nhiễu (Statistical Outlier Removal)
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(valid_verts)
            pcd.colors = o3d.utility.Vector3dVector(valid_colors)

            print("[Xử lý] Đang dọn sạch các vệt nhiễu dọc...")
            cl, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.0)
            pcd = pcd.select_by_index(ind)

            # 5. TẠO BỀ MẶT MESH CÓ MÀU
            print("[Mesh] Đang tính toán bề mặt Mesh...")
            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=30))
            pcd.orient_normals_towards_camera_location(camera_location=[0, 0, 0])
            
            # Sử dụng Ball Pivoting để tạo lưới tam giác
            distances = pcd.compute_nearest_neighbor_distance()
            avg_dist = np.mean(distances)
            radius = 2 * avg_dist
            mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
                pcd, o3d.utility.DoubleVector([radius, radius * 2])
            )

            # QUAN TRỌNG: Gán màu cho các đỉnh của Mesh
            mesh.vertex_colors = pcd.colors

            # 6. Lưu file PLY định dạng ASCII chuẩn (Có màu, có mặt)
            filename = "face_3d_colored_clean.ply"
            o3d.io.write_triangle_mesh(filename, mesh, write_ascii=True)
            
            full_path = os.path.abspath(filename)
            print(f"[THÀNH CÔNG] Đã lưu mô hình CÓ MÀU tại: {full_path}")
            
            # Tự động mở file bằng Windows 3D Viewer
            os.startfile(full_path)
            print("="*50 + "\n")

        except Exception as e:
            print(f"[LỖI] Quá trình thất bại: {e}")

    def run(self):
        pipeline_started = False
        try:
            profile = self.pipeline.start(self.config)
            pipeline_started = True
            
            # Tối ưu chế độ quét High Density
            device = profile.get_device()
            depth_sensor = device.query_sensors()[0]
            if depth_sensor.supports(rs.option.visual_preset):
                depth_sensor.set_option(rs.option.visual_preset, 4)

            window_name = 'SCANNER 3D - NHAN S DE CHUP MAT CO MAU'
            cv2.namedWindow(window_name)
            cv2.setMouseCallback(window_name, self.mouse_callback)

            print("\n* HƯỚNG DẪN: Ngồi cách camera 0.5m. Nhấn S để lưu Mesh CÓ MÀU DA THẬT.\n")

            while True:
                frames = self.pipeline.wait_for_frames()
                aligned_frames = self.align.process(frames)
                depth_frame = aligned_frames.get_depth_frame()
                color_frame = aligned_frames.get_color_frame()

                if not depth_frame or not color_frame: continue

                # Bộ lọc làm mịn
                depth_frame = self.spatial.process(depth_frame)
                depth_frame = self.temporal.process(depth_frame)
                depth_frame = self.hole_filling.process(depth_frame).as_depth_frame()

                color_image = np.asanyarray(color_frame.get_data())
                h, w = color_image.shape[:2]
                rgb_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
                results = self.mp_face_mesh.process(rgb_image)

                # Hiển thị khoảng cách hỗ trợ căn chỉnh
                dist_center = self.get_average_depth(depth_frame, w//2, h//2, h, w)
                color_dist = (0, 255, 0) if 0.4 <= dist_center <= 0.6 else (0, 0, 255)
                cv2.putText(color_image, f"Distance: {dist_center:.2f}m", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_dist, 2)

                if results.multi_face_landmarks:
                    for face_landmarks in results.multi_face_landmarks:
                        self.mp_drawing.draw_landmarks(
                            image=color_image, landmark_list=face_landmarks,
                            connections=self.FACEMESH_TESSELATION,
                            connection_drawing_spec=self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                        )

                cv2.imshow(window_name, color_image)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'): break
                elif key == ord('s'):
                    self.show_3d_snapshot(depth_frame, color_frame)
                elif key == ord('c'):
                    self.clicked_point = None

        except Exception as e:
            print(f"Lỗi: {e}")
        finally:
            if pipeline_started: self.pipeline.stop()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    scanner = RealSenseFaceScanner()
    scanner.run()


* HƯỚNG DẪN: Ngồi cách camera 0.5m. Nhấn S để lưu Mesh CÓ MÀU DA THẬT.


[3D SNAPSHOT] ĐANG TẠO MÔ HÌNH 3D CÓ MÀU SẮC...
[Xử lý] Đang dọn sạch các vệt nhiễu dọc...
[Mesh] Đang tính toán bề mặt Mesh...
[THÀNH CÔNG] Đã lưu mô hình CÓ MÀU tại: d:\FaceGrid3D\face_3d_colored_clean.ply



In [3]:
import pyrealsense2 as rs
import numpy as np
import cv2
import mediapipe as mp
import open3d as o3d
import os

class RealSenseFaceScanner:
    def __init__(self):
        # 1. Khởi tạo MediaPipe Face Mesh (Refine landmarks để lấy vùng mắt/môi chuẩn)
        self.mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.6,
            min_tracking_confidence=0.6
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.FACEMESH_TESSELATION = mp.solutions.face_mesh.FACEMESH_TESSELATION

        # 2. Khởi tạo RealSense D455
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        # D455 chạy ổn định nhất ở 640x480 để giảm nhiễu mép
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        self.align = rs.align(rs.stream.color)

        # 3. Các bộ lọc xử lý nhiễu
        self.spatial = rs.spatial_filter()
        self.temporal = rs.temporal_filter()
        self.hole_filling = rs.hole_filling_filter(2) 

        # Các landmark quan trọng để đo đạc sinh trắc học
        self.key_landmarks = {
            1: 'nose_tip',
            10: 'forehead',
            152: 'chin',
            129: 'left_nose_wing',
            358: 'right_nose_wing',
            468: 'right_iris',
            473: 'left_iris'
        }

        self.clicked_point = None

    def mouse_callback(self, event, x, y, flags, param):
        """ Xử lý sự kiện click chuột để lấy tọa độ pixel """
        if event == cv2.EVENT_LBUTTONDOWN:
            self.clicked_point = (x, y)
            print(f"[Mouse Event] Tọa độ chọn: ({x}, {y})")

    def get_average_depth(self, depth_frame, x, y, h, w, window_size=5):
        """ Tính độ sâu trung bình trong vùng nhỏ """
        depths = []
        half = window_size // 2
        for dy in range(-half, half + 1):
            for dx in range(-half, half + 1):
                nx, ny = x + dx, y + dy
                if 0 <= nx < w and 0 <= ny < h:
                    d = depth_frame.get_distance(nx, ny)
                    if d > 0: depths.append(d)
        return np.mean(depths) if depths else 0.0

    def show_3d_snapshot(self, depth_frame, color_frame, face_landmarks):
        """ 
        PHIÊN BẢN TỐI ƯU CHO MESHLAB:
        - Sử dụng mặt nạ MediaPipe để lọc nhiễu 100%.
        - Căn giữa dữ liệu (Normalization) để MeshLab không bị tối thui.
        - Tính toán lại pháp tuyến và gán màu sắc chuẩn xác cho Mesh.
        """
        try:
            print("\n" + "="*50)
            print("[3D SNAPSHOT] ĐANG TẠO MESH CHUẨN MESHLAB...")
            
            # 1. Chuẩn bị dữ liệu ảnh
            color_image = np.asanyarray(color_frame.get_data())
            h, w = color_image.shape[:2]
            color_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

            # 2. TẠO MẶT NẠ (MASK) TỪ LANDMARKS ĐỂ LỌC VÙNG MẶT
            face_oval_indices = [
                10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 
                397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 
                172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109
            ]
            points = []
            for idx in face_oval_indices:
                lm = face_landmarks.landmark[idx]
                points.append([int(lm.x * w), int(lm.y * h)])
            
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(mask, [np.array(points)], 255)

            # 3. Tính toán mây điểm (Point Cloud)
            pc = rs.pointcloud()
            pc.map_to(color_frame)
            points_3d_all = pc.calculate(depth_frame)
            v = points_3d_all.get_vertices()
            verts = np.asanyarray(v).view(np.float32).reshape(h, w, 3)
            
            # 4. TRÍCH XUẤT ĐIỂM DỰA TRÊN MẶT NẠ
            valid_indices = np.where((mask > 0) & (verts[:, :, 2] > 0.1) & (verts[:, :, 2] < 1.0))
            final_verts = verts[valid_indices]
            final_colors = color_rgb[valid_indices]

            if len(final_verts) < 2000:
                print("[LỖI] Không đủ dữ liệu mặt. Hãy ngồi gần camera hơn!")
                return

            # 5. Tạo Point Cloud và Xử lý Hệ tọa độ
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(final_verts)
            pcd.colors = o3d.utility.Vector3dVector(final_colors)

            # Lọc nhiễu thống kê
            pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.5)

            # CĂN GIỮA DỮ LIỆU (Normalization): Giúp MeshLab tìm thấy vật thể ngay tại gốc tọa độ
            pcd.translate(-pcd.get_center())

            # XOAY HƯỚNG (Fix Flip): Xoay 180 độ quanh trục X để mặt đứng thẳng
            R = pcd.get_rotation_matrix_from_axis_angle([np.pi, 0, 0])
            pcd.rotate(R, center=(0, 0, 0))

            # ƯỚC TÍNH PHÁP TUYẾN
            print("[Xử lý] Đang tính toán pháp tuyến bề mặt...")
            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=30))
            # Hướng pháp tuyến về phía camera (bây giờ camera ảo coi như ở phía trước trục Z)
            pcd.orient_normals_towards_camera_location(camera_location=[0, 0, 10])

            # 6. DỰNG HÌNH BỀ MẶT (Triangle Mesh)
            print("[Mesh] Đang dựng lưới bề mặt (Ball Pivoting)...")
            distances = pcd.compute_nearest_neighbor_distance()
            avg_dist = np.mean(distances)
            radius = 2 * avg_dist
            mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
                pcd, o3d.utility.DoubleVector([radius, radius * 2])
            )

            # GÁN MÀU SẮC CHO MESH: Đảm bảo MeshLab hiển thị màu da thật
            mesh.vertex_colors = pcd.colors

            # 7. LƯU FILE CHUẨN
            filename = "face_mesh_meshlab_ready.ply"
            o3d.io.write_triangle_mesh(filename, mesh)
            
            full_path = os.path.abspath(filename)
            print(f"[THÀNH CÔNG] File 3D chuẩn đã sẵn sàng: {full_path}")
            
            # Tự động mở file
            os.startfile(full_path)
            print("="*50 + "\n")

        except Exception as e:
            print(f"[LỖI] {e}")

    def run(self):
        pipeline_started = False
        try:
            profile = self.pipeline.start(self.config)
            pipeline_started = True
            
            # Tối ưu chế độ quét High Density cho Face Scanning
            device = profile.get_device()
            depth_sensor = device.query_sensors()[0]
            if depth_sensor.supports(rs.option.visual_preset):
                depth_sensor.set_option(rs.option.visual_preset, 4)

            window_name = 'DESLAB 3D SCANNER - PRO EDITION'
            cv2.namedWindow(window_name)
            cv2.setMouseCallback(window_name, self.mouse_callback)

            current_landmarks = None

            print("\n>>> HƯỚNG DẪN: Đảm bảo Distance hiển thị màu Xanh. Nhấn 'S' để chụp Mesh chuẩn MeshLab.")

            while True:
                frames = self.pipeline.wait_for_frames()
                aligned_frames = self.align.process(frames)
                depth_frame = aligned_frames.get_depth_frame()
                color_frame = aligned_frames.get_color_frame()

                if not depth_frame or not color_frame: continue

                # Áp dụng bộ lọc Depth
                depth_frame = self.spatial.process(depth_frame)
                depth_frame = self.temporal.process(depth_frame)
                depth_frame = self.hole_filling.process(depth_frame).as_depth_frame()

                color_image = np.asanyarray(color_frame.get_data())
                h, w = color_image.shape[:2]
                rgb_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
                results = self.mp_face_mesh.process(rgb_image)

                # Hiển thị thước đo khoảng cách
                d_center = self.get_average_depth(depth_frame, w//2, h//2, h, w)
                color_status = (0, 255, 0) if 0.45 <= d_center <= 0.6 else (0, 0, 255)
                cv2.putText(color_image, f"Distance: {d_center:.2f}m", (10, 40), 0, 0.7, color_status, 2)

                if results.multi_face_landmarks:
                    current_landmarks = results.multi_face_landmarks[0]
                    # Vẽ lưới Face Mesh để người dùng căn chỉnh
                    self.mp_drawing.draw_landmarks(
                        image=color_image, landmark_list=current_landmarks,
                        connections=self.FACEMESH_TESSELATION,
                        connection_drawing_spec=self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                    )

                    # Tính toán đo đạc IPD (mm) và Chiều cao mặt (cm) thời gian thực
                    color_intrinsics = color_frame.profile.as_video_stream_profile().intrinsics
                    points_3d = {}
                    for idx, name in self.key_landmarks.items():
                        lm = current_landmarks.landmark[idx]
                        u, v = int(lm.x * w), int(lm.y * h)
                        depth = self.get_average_depth(depth_frame, u, v, h, w)
                        if depth > 0:
                            pt = rs.rs2_deproject_pixel_to_point(color_intrinsics, [u, v], depth)
                            points_3d[name] = np.array(pt)

                    y_off = 70
                    if 'left_iris' in points_3d and 'right_iris' in points_3d:
                        ipd = np.linalg.norm(points_3d['left_iris'] - points_3d['right_iris']) * 1000
                        cv2.putText(color_image, f"IPD: {ipd:.1f} mm", (10, y_off), 0, 0.6, (255, 255, 0), 2)
                        y_off += 25
                    
                    if 'forehead' in points_3d and 'chin' in points_3d:
                        f_h = np.linalg.norm(points_3d['forehead'] - points_3d['chin']) * 100
                        cv2.putText(color_image, f"Face Height: {f_h:.1f} cm", (10, y_off), 0, 0.6, (255, 255, 0), 2)

                # Hiển thị điểm click chuột
                if self.clicked_point:
                    cx, cy = self.clicked_point
                    d_click = self.get_average_depth(depth_frame, cx, cy, h, w)
                    cv2.drawMarker(color_image, (cx, cy), (0, 0, 255), cv2.MARKER_CROSS, 20, 2)
                    if d_click > 0:
                        cv2.putText(color_image, f"{d_click:.3f}m", (cx + 10, cy - 5), 0, 0.6, (0, 255, 255), 2)

                cv2.imshow(window_name, color_image)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'): break
                elif key == ord('s') and current_landmarks:
                    self.show_3d_snapshot(depth_frame, color_frame, current_landmarks)
                elif key == ord('c'):
                    self.clicked_point = None

        except Exception as e:
            print(f"Lỗi: {e}")
        finally:
            if pipeline_started: self.pipeline.stop()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    scanner = RealSenseFaceScanner()
    scanner.run()


>>> HƯỚNG DẪN: Đảm bảo Distance hiển thị màu Xanh. Nhấn 'S' để chụp Mesh chuẩn MeshLab.
[Mouse Event] Tọa độ chọn: (222, 130)
[Mouse Event] Tọa độ chọn: (233, 130)
[Mouse Event] Tọa độ chọn: (258, 130)
[Mouse Event] Tọa độ chọn: (295, 160)
[Mouse Event] Tọa độ chọn: (361, 201)
[Mouse Event] Tọa độ chọn: (393, 230)
[Mouse Event] Tọa độ chọn: (280, 235)
[Mouse Event] Tọa độ chọn: (124, 202)
[Mouse Event] Tọa độ chọn: (242, 186)


In [4]:
import pyrealsense2 as rs
import numpy as np
import cv2
import mediapipe as mp
import open3d as o3d
import os

class RealSenseFaceScanner:
    def __init__(self):
        # 1. Khởi tạo MediaPipe Face Mesh (Refine landmarks để lấy vùng mắt/môi chuẩn)
        self.mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.6,
            min_tracking_confidence=0.6
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.FACEMESH_TESSELATION = mp.solutions.face_mesh.FACEMESH_TESSELATION

        # 2. Khởi tạo RealSense D455
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        # D455 chạy ổn định nhất ở 640x480 để giảm nhiễu mép
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        self.align = rs.align(rs.stream.color)

        # 3. Các bộ lọc xử lý nhiễu
        self.spatial = rs.spatial_filter()
        self.temporal = rs.temporal_filter()
        self.hole_filling = rs.hole_filling_filter(2) 

        # [UPDATE] Các landmark quan trọng cho đo đạc sinh trắc học (mm accuracy)
        self.key_landmarks = {
            1: 'nose_tip',
            10: 'forehead',
            152: 'chin',
            129: 'left_nose_wing',
            358: 'right_nose_wing',
            468: 'right_iris',
            473: 'left_iris',
            50: 'left_cheek',
            280: 'right_cheek',
            234: 'left_ear',
            454: 'right_ear'
        }

        self.clicked_point = None

    def mouse_callback(self, event, x, y, flags, param):
        """ Xử lý sự kiện click chuột để lấy tọa độ pixel """
        if event == cv2.EVENT_LBUTTONDOWN:
            self.clicked_point = (x, y)
            print(f"[Mouse Event] Tọa độ chọn: ({x}, {y})")

    def get_average_depth(self, depth_frame, x, y, h, w, window_size=5):
        """ Tính độ sâu trung bình trong vùng 5x5 để đảm bảo độ chính xác milimet """
        depths = []
        half = window_size // 2
        for dy in range(-half, half + 1):
            for dx in range(-half, half + 1):
                nx, ny = x + dx, y + dy
                if 0 <= nx < w and 0 <= ny < h:
                    d = depth_frame.get_distance(nx, ny)
                    if d > 0: depths.append(d)
        return np.mean(depths) if depths else 0.0

    def show_3d_snapshot(self, depth_frame, color_frame, face_landmarks):
        """ PHIÊN BẢN TỐI ƯU CHO MESHLAB VỚI MẶT NẠ LANDMARK """
        try:
            print("\n" + "="*50)
            print("[3D SNAPSHOT] ĐANG TẠO MESH CHUẨN MESHLAB...")
            
            color_image = np.asanyarray(color_frame.get_data())
            h, w = color_image.shape[:2]
            color_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

            # TẠO MẶT NẠ (MASK) TỪ LANDMARKS
            face_oval_indices = [
                10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 
                397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 
                172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109
            ]
            points = []
            for idx in face_oval_indices:
                lm = face_landmarks.landmark[idx]
                points.append([int(lm.x * w), int(lm.y * h)])
            
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(mask, [np.array(points)], 255)

            pc = rs.pointcloud()
            pc.map_to(color_frame)
            points_3d_all = pc.calculate(depth_frame)
            v = points_3d_all.get_vertices()
            verts = np.asanyarray(v).view(np.float32).reshape(h, w, 3)
            
            valid_indices = np.where((mask > 0) & (verts[:, :, 2] > 0.1) & (verts[:, :, 2] < 1.0))
            final_verts = verts[valid_indices]
            final_colors = color_rgb[valid_indices]

            if len(final_verts) < 2000:
                print("[LỖI] Không đủ dữ liệu mặt.")
                return

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(final_verts)
            pcd.colors = o3d.utility.Vector3dVector(final_colors)

            pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.5)
            pcd.translate(-pcd.get_center())
            R = pcd.get_rotation_matrix_from_axis_angle([np.pi, 0, 0])
            pcd.rotate(R, center=(0, 0, 0))

            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=30))
            pcd.orient_normals_towards_camera_location(camera_location=[0, 0, 10])

            distances = pcd.compute_nearest_neighbor_distance()
            avg_dist = np.mean(distances)
            radius = 2 * avg_dist
            mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
                pcd, o3d.utility.DoubleVector([radius, radius * 2])
            )
            mesh.vertex_colors = pcd.colors

            filename = "face_mesh_biometrics.ply"
            o3d.io.write_triangle_mesh(filename, mesh)
            full_path = os.path.abspath(filename)
            print(f"[THÀNH CÔNG] File đã lưu tại: {full_path}")
            os.startfile(full_path)

        except Exception as e:
            print(f"[LỖI] {e}")

    def run(self):
        pipeline_started = False
        try:
            profile = self.pipeline.start(self.config)
            pipeline_started = True
            
            device = profile.get_device()
            depth_sensor = device.query_sensors()[0]
            if depth_sensor.supports(rs.option.visual_preset):
                depth_sensor.set_option(rs.option.visual_preset, 4)

            window_name = '3D BIOMETRIC SCANNER - D455'
            cv2.namedWindow(window_name)
            cv2.setMouseCallback(window_name, self.mouse_callback)

            current_landmarks = None

            print("\n>>> HƯỚNG DẪN: Distance màu Xanh (0.5m) để đạt độ chính xác Milimet.")

            while True:
                frames = self.pipeline.wait_for_frames()
                aligned_frames = self.align.process(frames)
                depth_frame = aligned_frames.get_depth_frame()
                color_frame = aligned_frames.get_color_frame()

                if not depth_frame or not color_frame: continue

                depth_frame = self.spatial.process(depth_frame)
                depth_frame = self.temporal.process(depth_frame)
                depth_frame = self.hole_filling.process(depth_frame).as_depth_frame()

                color_image = np.asanyarray(color_frame.get_data())
                h, w = color_image.shape[:2]
                rgb_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
                results = self.mp_face_mesh.process(rgb_image)

                # Vẽ khung Dashboard bán trong suốt
                overlay = color_image.copy()
                cv2.rectangle(overlay, (5, 5), (280, 240), (0, 0, 0), -1)
                cv2.addWeighted(overlay, 0.6, color_image, 0.4, 0, color_image)

                # Khoảng cách trung tâm
                d_center = self.get_average_depth(depth_frame, w//2, h//2, h, w)
                color_status = (0, 255, 0) if 0.45 <= d_center <= 0.6 else (0, 0, 255)
                cv2.putText(color_image, f"Distance: {d_center:.2f}m", (15, 30), 0, 0.6, color_status, 2)

                if results.multi_face_landmarks:
                    current_landmarks = results.multi_face_landmarks[0]
                    self.mp_drawing.draw_landmarks(
                        image=color_image, landmark_list=current_landmarks,
                        connections=self.FACEMESH_TESSELATION,
                        connection_drawing_spec=self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                    )

                    color_intrinsics = color_frame.profile.as_video_stream_profile().intrinsics
                    points_3d = {}
                    for idx, name in self.key_landmarks.items():
                        lm = current_landmarks.landmark[idx]
                        u, v = int(lm.x * w), int(lm.y * h)
                        depth = self.get_average_depth(depth_frame, u, v, h, w)
                        if depth > 0:
                            pt = rs.rs2_deproject_pixel_to_point(color_intrinsics, [u, v], depth)
                            points_3d[name] = np.array(pt)

                    # LOGIC ĐO ĐẠC CHI TIẾT (mm/cm)
                    y_off = 65
                    txt_color = (255, 255, 255)
                    
                    # 1. IPD (mm)
                    if 'left_iris' in points_3d and 'right_iris' in points_3d:
                        val = np.linalg.norm(points_3d['left_iris'] - points_3d['right_iris']) * 1000
                        cv2.putText(color_image, f"IPD: {val:.1f} mm", (15, y_off), 0, 0.55, (255, 255, 0), 1)
                        y_off += 25
                    
                    # 2. Face Height (cm)
                    if 'forehead' in points_3d and 'chin' in points_3d:
                        val = np.linalg.norm(points_3d['forehead'] - points_3d['chin']) * 100
                        cv2.putText(color_image, f"Height: {val:.1f} cm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                    # 3. Face Width (mm) - Ear to Ear
                    if 'left_ear' in points_3d and 'right_ear' in points_3d:
                        val = np.linalg.norm(points_3d['left_ear'] - points_3d['right_ear']) * 1000
                        cv2.putText(color_image, f"Face Width: {val:.1f} mm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                    # 4. Cheek Width (mm)
                    if 'left_cheek' in points_3d and 'right_cheek' in points_3d:
                        val = np.linalg.norm(points_3d['left_cheek'] - points_3d['right_cheek']) * 1000
                        cv2.putText(color_image, f"Cheek Width: {val:.1f} mm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                    # 5. Nose Width (mm)
                    if 'left_nose_wing' in points_3d and 'right_nose_wing' in points_3d:
                        val = np.linalg.norm(points_3d['left_nose_wing'] - points_3d['right_nose_wing']) * 1000
                        cv2.putText(color_image, f"Nose Width: {val:.1f} mm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                # Click chuột đo tự do
                if self.clicked_point:
                    cx, cy = self.clicked_point
                    d_click = self.get_average_depth(depth_frame, cx, cy, h, w)
                    cv2.drawMarker(color_image, (cx, cy), (0, 0, 255), cv2.MARKER_CROSS, 20, 2)
                    if d_click > 0:
                        cv2.putText(color_image, f"Depth: {d_click:.3f}m", (cx + 10, cy - 5), 0, 0.6, (0, 255, 255), 2)

                cv2.imshow(window_name, color_image)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'): break
                elif key == ord('s') and current_landmarks:
                    self.show_3d_snapshot(depth_frame, color_frame, current_landmarks)
                elif key == ord('c'):
                    self.clicked_point = None

        except Exception as e:
            print(f"Lỗi: {e}")
        finally:
            if pipeline_started: self.pipeline.stop()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    scanner = RealSenseFaceScanner()
    scanner.run()


>>> HƯỚNG DẪN: Distance màu Xanh (0.5m) để đạt độ chính xác Milimet.
[Mouse Event] Tọa độ chọn: (394, 254)
[Mouse Event] Tọa độ chọn: (383, 202)
[Mouse Event] Tọa độ chọn: (437, 212)
[Mouse Event] Tọa độ chọn: (363, 156)
[Mouse Event] Tọa độ chọn: (379, 150)
[Mouse Event] Tọa độ chọn: (344, 120)
[Mouse Event] Tọa độ chọn: (337, 121)
[Mouse Event] Tọa độ chọn: (330, 121)
[Mouse Event] Tọa độ chọn: (342, 109)
[Mouse Event] Tọa độ chọn: (347, 145)
[Mouse Event] Tọa độ chọn: (350, 139)
[Mouse Event] Tọa độ chọn: (311, 77)
[Mouse Event] Tọa độ chọn: (314, 78)
[Mouse Event] Tọa độ chọn: (501, 213)
[Mouse Event] Tọa độ chọn: (287, 276)
[Mouse Event] Tọa độ chọn: (590, 67)


KeyboardInterrupt: 

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import mediapipe as mp
import open3d as o3d
import os

class RealSenseFaceScanner:
    def __init__(self):
        # 1. Khởi tạo MediaPipe Face Mesh (Refine landmarks để lấy vùng mắt/môi chuẩn)
        self.mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.6,
            min_tracking_confidence=0.6
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.FACEMESH_TESSELATION = mp.solutions.face_mesh.FACEMESH_TESSELATION

        # 2. Khởi tạo RealSense D455
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        # D455 chạy ổn định nhất ở 640x480 để giảm nhiễu mép
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        self.align = rs.align(rs.stream.color)

        # 3. Các bộ lọc xử lý nhiễu
        self.spatial = rs.spatial_filter()
        self.temporal = rs.temporal_filter()
        self.hole_filling = rs.hole_filling_filter(2) 

        # Các landmark quan trọng cho đo đạc sinh trắc học (mm accuracy)
        self.key_landmarks = {
            1: 'nose_tip',
            10: 'forehead',
            152: 'chin',
            129: 'left_nose_wing',
            358: 'right_nose_wing',
            468: 'right_iris',
            473: 'left_iris',
            50: 'left_cheek',
            280: 'right_cheek',
            234: 'left_ear',
            454: 'right_ear'
        }

        self.clicked_point = None

    def mouse_callback(self, event, x, y, flags, param):
        """ Xử lý sự kiện click chuột để lấy tọa độ pixel """
        if event == cv2.EVENT_LBUTTONDOWN:
            self.clicked_point = (x, y)
            print(f"[Mouse Event] Tọa độ chọn: ({x}, {y})")

    def get_average_depth(self, depth_frame, x, y, h, w, window_size=5):
        """ Tính độ sâu trung bình trong vùng 5x5 để đảm bảo độ chính xác milimet """
        depths = []
        half = window_size // 2
        for dy in range(-half, half + 1):
            for dx in range(-half, half + 1):
                nx, ny = x + dx, y + dy
                if 0 <= nx < w and 0 <= ny < h:
                    d = depth_frame.get_distance(nx, ny)
                    if d > 0: depths.append(d)
        return np.mean(depths) if depths else 0.0

    def show_3d_snapshot(self, depth_frame, color_frame, face_landmarks):
        """ PHIÊN BẢN TỐI ƯU CHO MESHLAB VỚI MẶT NẠ LANDMARK """
        try:
            print("\n" + "="*50)
            print("[3D SNAPSHOT] ĐANG TẠO MESH CHUẨN MESHLAB...")
            
            color_image = np.asanyarray(color_frame.get_data())
            h, w = color_image.shape[:2]
            color_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

            # TẠO MẶT NẠ (MASK) TỪ LANDMARKS
            face_oval_indices = [
                10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 
                397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 
                172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109
            ]
            points = []
            for idx in face_oval_indices:
                lm = face_landmarks.landmark[idx]
                points.append([int(lm.x * w), int(lm.y * h)])
            
            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(mask, [np.array(points)], 255)

            pc = rs.pointcloud()
            pc.map_to(color_frame)
            points_3d_all = pc.calculate(depth_frame)
            v = points_3d_all.get_vertices()
            verts = np.asanyarray(v).view(np.float32).reshape(h, w, 3)
            
            valid_indices = np.where((mask > 0) & (verts[:, :, 2] > 0.1) & (verts[:, :, 2] < 1.0))
            final_verts = verts[valid_indices]
            final_colors = color_rgb[valid_indices]

            if len(final_verts) < 2000:
                print("[LỖI] Không đủ dữ liệu mặt.")
                return

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(final_verts)
            pcd.colors = o3d.utility.Vector3dVector(final_colors)

            pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.5)
            pcd.translate(-pcd.get_center())
            R = pcd.get_rotation_matrix_from_axis_angle([np.pi, 0, 0])
            pcd.rotate(R, center=(0, 0, 0))

            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.03, max_nn=30))
            pcd.orient_normals_towards_camera_location(camera_location=[0, 0, 10])

            distances = pcd.compute_nearest_neighbor_distance()
            avg_dist = np.mean(distances)
            radius = 2 * avg_dist
            mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
                pcd, o3d.utility.DoubleVector([radius, radius * 2])
            )
            mesh.vertex_colors = pcd.colors

            filename = "face_mesh_biometrics.ply"
            o3d.io.write_triangle_mesh(filename, mesh)
            full_path = os.path.abspath(filename)
            print(f"[THÀNH CÔNG] File đã lưu tại: {full_path}")
            os.startfile(full_path)

        except Exception as e:
            print(f"[LỖI] {e}")

    def run(self):
        pipeline_started = False
        try:
            profile = self.pipeline.start(self.config)
            pipeline_started = True
            
            device = profile.get_device()
            depth_sensor = device.query_sensors()[0]
            if depth_sensor.supports(rs.option.visual_preset):
                depth_sensor.set_option(rs.option.visual_preset, 4)

            window_name = '3D BIOMETRIC SCANNER - D455'
            cv2.namedWindow(window_name)
            cv2.setMouseCallback(window_name, self.mouse_callback)

            current_landmarks = None

            print("\n>>> HƯỚNG DẪN: Distance màu Xanh (0.5m) để đạt độ chính xác Milimet.")

            while True:
                frames = self.pipeline.wait_for_frames()
                aligned_frames = self.align.process(frames)
                depth_frame = aligned_frames.get_depth_frame()
                color_frame = aligned_frames.get_color_frame()

                if not depth_frame or not color_frame: continue

                depth_frame = self.spatial.process(depth_frame)
                depth_frame = self.temporal.process(depth_frame)
                depth_frame = self.hole_filling.process(depth_frame).as_depth_frame()

                color_image = np.asanyarray(color_frame.get_data())
                h, w = color_image.shape[:2]
                rgb_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
                results = self.mp_face_mesh.process(rgb_image)

                # Vẽ khung Dashboard bán trong suốt cho thông số
                overlay = color_image.copy()
                cv2.rectangle(overlay, (5, 5), (280, 240), (0, 0, 0), -1)
                cv2.addWeighted(overlay, 0.6, color_image, 0.4, 0, color_image)

                # Khoảng cách trung tâm
                d_center = self.get_average_depth(depth_frame, w//2, h//2, h, w)
                color_status = (0, 255, 0) if 0.45 <= d_center <= 0.6 else (0, 0, 255)
                cv2.putText(color_image, f"Distance: {d_center:.2f}m", (15, 30), 0, 0.6, color_status, 2)

                if results.multi_face_landmarks:
                    current_landmarks = results.multi_face_landmarks[0]
                    
                    # [UPDATE] ẨN CHẤM ĐỎ: Sử dụng landmark_drawing_spec với độ dày và bán kính = 0
                    self.mp_drawing.draw_landmarks(
                        image=color_image, 
                        landmark_list=current_landmarks,
                        connections=self.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None, # MediaPipe mặc định vẽ chấm nếu để None, ta sẽ ghi đè bên dưới
                        connection_drawing_spec=self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                    )
                    # Một số phiên bản yêu cầu tham số landmark_drawing_spec cụ thể để ẩn điểm:
                    # Để ẩn hẳn các chấm đỏ, ta có thể dùng DrawingSpec với circle_radius=0
                    invisible_spec = self.mp_drawing.DrawingSpec(thickness=0, circle_radius=0)
                    self.mp_drawing.draw_landmarks(
                        image=color_image,
                        landmark_list=current_landmarks,
                        connections=self.FACEMESH_TESSELATION,
                        landmark_drawing_spec=invisible_spec,
                        connection_drawing_spec=self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)
                    )

                    color_intrinsics = color_frame.profile.as_video_stream_profile().intrinsics
                    points_3d = {}
                    for idx, name in self.key_landmarks.items():
                        lm = current_landmarks.landmark[idx]
                        u, v = int(lm.x * w), int(lm.y * h)
                        depth = self.get_average_depth(depth_frame, u, v, h, w)
                        if depth > 0:
                            pt = rs.rs2_deproject_pixel_to_point(color_intrinsics, [u, v], depth)
                            points_3d[name] = np.array(pt)

                    # LOGIC ĐO ĐẠC CHI TIẾT (mm/cm)
                    y_off = 65
                    txt_color = (255, 255, 255)
                    
                    if 'left_iris' in points_3d and 'right_iris' in points_3d:
                        val = np.linalg.norm(points_3d['left_iris'] - points_3d['right_iris']) * 1000
                        cv2.putText(color_image, f"IPD: {val:.1f} mm", (15, y_off), 0, 0.55, (255, 255, 0), 1)
                        y_off += 25
                    
                    if 'forehead' in points_3d and 'chin' in points_3d:
                        val = np.linalg.norm(points_3d['forehead'] - points_3d['chin']) * 100
                        cv2.putText(color_image, f"Height: {val:.1f} cm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                    if 'left_ear' in points_3d and 'right_ear' in points_3d:
                        val = np.linalg.norm(points_3d['left_ear'] - points_3d['right_ear']) * 1000
                        cv2.putText(color_image, f"Face Width: {val:.1f} mm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                    if 'left_cheek' in points_3d and 'right_cheek' in points_3d:
                        val = np.linalg.norm(points_3d['left_cheek'] - points_3d['right_cheek']) * 1000
                        cv2.putText(color_image, f"Cheek Width: {val:.1f} mm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                    if 'left_nose_wing' in points_3d and 'right_nose_wing' in points_3d:
                        val = np.linalg.norm(points_3d['left_nose_wing'] - points_3d['right_nose_wing']) * 1000
                        cv2.putText(color_image, f"Nose Width: {val:.1f} mm", (15, y_off), 0, 0.55, txt_color, 1)
                        y_off += 25

                # Click chuột đo tự do
                if self.clicked_point:
                    cx, cy = self.clicked_point
                    d_click = self.get_average_depth(depth_frame, cx, cy, h, w)
                    cv2.drawMarker(color_image, (cx, cy), (0, 0, 255), cv2.MARKER_CROSS, 20, 2)
                    if d_click > 0:
                        cv2.putText(color_image, f"Depth: {d_click:.3f}m", (cx + 10, cy - 5), 0, 0.6, (0, 255, 255), 2)

                cv2.imshow(window_name, color_image)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'): break
                elif key == ord('s') and current_landmarks:
                    self.show_3d_snapshot(depth_frame, color_frame, current_landmarks)
                elif key == ord('c'):
                    self.clicked_point = None

        except Exception as e:
            print(f"Lỗi: {e}")
        finally:
            if pipeline_started: self.pipeline.stop()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    scanner = RealSenseFaceScanner()
    scanner.run()
    


>>> HƯỚNG DẪN: Distance màu Xanh (0.5m) để đạt độ chính xác Milimet.


In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import open3d as o3d
import os
from scipy.spatial import Delaunay
from ultralytics import YOLO


class RealSenseYOLOScanner:
    def __init__(self, model_name="yolov8n-seg.pt", confidence=0.5):
        """
        model_name: YOLOv8-seg model (yolov8n-seg / s-seg / m-seg / l-seg / x-seg .pt)
                    PHẢI dùng model -seg để có segmentation mask!
        confidence: Ngưỡng confidence tối thiểu
        """
        # ========== YOLOv8-Seg ==========
        print("[YOLO-SEG] Đang tải model...")
        self.model = YOLO(model_name)
        self.confidence = confidence
        print(f"[YOLO-SEG] Model '{model_name}' đã sẵn sàng!")

        # ========== RealSense D455 ==========
        self.pipeline = rs.pipeline()
        self.config = rs.config()
        self.config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        self.config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
        self.align = rs.align(rs.stream.color)

        # Bộ lọc nhiễu depth
        self.spatial = rs.spatial_filter()
        self.temporal = rs.temporal_filter()
        self.hole_filling = rs.hole_filling_filter(2)

        # Khoảng depth quan tâm (mét)
        self.min_depth = 0.15
        self.max_depth = 2.0

        # Poisson reconstruction depth
        self.poisson_depth = 9

        # Mesh wireframe settings
        self.mesh_grid_step = 8       # Khoảng cách giữa các điểm lưới (pixel), nhỏ hơn = dày hơn
        self.mesh_line_thickness = 1  # Độ dày đường lưới
        self.show_mesh_wireframe = True  # Bật/tắt lưới mesh

        # State
        self.clicked_point = None
        self.selected_object = None

        # Màu cố định cho từng class
        np.random.seed(42)
        self.class_colors = {
            i: tuple(int(c) for c in np.random.randint(80, 255, 3))
            for i in range(80)
        }

    # ==========================================
    #  VẼ LƯỚI MESH WIREFRAME LÊN VẬT THỂ
    #  (Tương tự FACEMESH_TESSELATION của MediaPipe)
    # ==========================================
    def draw_mesh_wireframe(self, image, det, depth_frame, h, w):
        """
        Vẽ lưới tam giác (Delaunay tesselation) lên vật thể trong ảnh camera.
        Giống cách MediaPipe vẽ FACEMESH_TESSELATION cho khuôn mặt.

        Logic:
        1. Sample đều các điểm trong segmentation mask
        2. Lọc điểm có depth hợp lệ
        3. Delaunay triangulation trên toạ độ 2D
        4. Vẽ các cạnh tam giác, tô màu theo depth
        """
        seg_mask = det['seg_mask']
        if seg_mask is None:
            return

        color = self.class_colors.get(det['cls_id'], (0, 255, 0))
        bx, by, bw, bh = det['bbox']
        step = self.mesh_grid_step

        # === 1. Sample điểm đều trong mask ===
        points_2d = []
        points_depth = []

        for py in range(by, min(by + bh, h), step):
            for px in range(bx, min(bx + bw, w), step):
                if seg_mask[py, px] > 0:
                    d = depth_frame.get_distance(px, py)
                    if self.min_depth < d < self.max_depth:
                        points_2d.append([px, py])
                        points_depth.append(d)

        # Thêm các điểm contour để lưới bám sát viền vật thể
        contours, _ = cv2.findContours(seg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            for i in range(0, len(cnt), max(1, len(cnt) // 60)):
                px, py = cnt[i][0]
                if 0 <= px < w and 0 <= py < h:
                    d = depth_frame.get_distance(int(px), int(py))
                    if self.min_depth < d < self.max_depth:
                        points_2d.append([int(px), int(py)])
                        points_depth.append(d)

        if len(points_2d) < 10:
            return

        points_2d = np.array(points_2d)
        points_depth = np.array(points_depth)

        # === 2. Delaunay triangulation (giống tesselation) ===
        try:
            tri = Delaunay(points_2d)
        except Exception:
            return

        # === 3. Tính depth range để map màu ===
        d_min = det['min_depth'] if det['min_depth'] > 0 else points_depth.min()
        d_max = det['max_depth'] if det['max_depth'] > 0 else points_depth.max()
        d_range = d_max - d_min if d_max > d_min else 0.01

        # === 4. Vẽ các tam giác ===
        for simplex in tri.simplices:
            p0 = points_2d[simplex[0]]
            p1 = points_2d[simplex[1]]
            p2 = points_2d[simplex[2]]

            # Kiểm tra tâm tam giác nằm trong mask (loại bỏ tam giác ngoài vật thể)
            cx_tri = int((p0[0] + p1[0] + p2[0]) / 3)
            cy_tri = int((p0[1] + p1[1] + p2[1]) / 3)
            if 0 <= cx_tri < w and 0 <= cy_tri < h:
                if seg_mask[cy_tri, cx_tri] == 0:
                    continue

            # Depth trung bình của tam giác → màu
            avg_d = (points_depth[simplex[0]] + points_depth[simplex[1]] + points_depth[simplex[2]]) / 3
            t = np.clip((avg_d - d_min) / d_range, 0, 1)

            # Gradient màu: gần (xanh lá sáng) → xa (xanh lá tối)
            r = int(color[0] * (1 - t * 0.6))
            g = int(color[1] * (1 - t * 0.6))
            b = int(color[2] * (1 - t * 0.6))
            line_color = (r, g, b)

            # Vẽ 3 cạnh tam giác
            pts = [tuple(p0), tuple(p1), tuple(p2)]
            cv2.line(image, pts[0], pts[1], line_color, self.mesh_line_thickness, cv2.LINE_AA)
            cv2.line(image, pts[1], pts[2], line_color, self.mesh_line_thickness, cv2.LINE_AA)
            cv2.line(image, pts[2], pts[0], line_color, self.mesh_line_thickness, cv2.LINE_AA)

    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.clicked_point = (x, y)
            if hasattr(self, '_current_detections'):
                for det in self._current_detections:
                    if det['seg_mask'] is not None and det['seg_mask'][y, x] > 0:
                        self.selected_object = det
                        print(f"[SELECT] Đã chọn: {det['label']} "
                              f"(conf: {det['conf']:.0%}, depth: {det['avg_depth']:.2f}m, "
                              f"mask pixels: {det['mask_area']})")
                        return
                    bx, by2, bw, bh = det['bbox']
                    if bx <= x <= bx + bw and by2 <= y <= by2 + bh:
                        self.selected_object = det
                        print(f"[SELECT] Đã chọn (bbox): {det['label']}")
                        return
            print(f"[Mouse] ({x}, {y}) - Không trúng vật thể nào")
            self.selected_object = None

    def get_average_depth(self, depth_frame, x, y, h, w, window_size=5):
        depths = []
        half = window_size // 2
        for dy in range(-half, half + 1):
            for dx in range(-half, half + 1):
                nx, ny = x + dx, y + dy
                if 0 <= nx < w and 0 <= ny < h:
                    d = depth_frame.get_distance(nx, ny)
                    if d > 0:
                        depths.append(d)
        return np.mean(depths) if depths else 0.0

    def detect_objects(self, color_image, depth_frame, h, w):
        results = self.model(color_image, conf=self.confidence, verbose=False)
        detections = []

        if not results or len(results) == 0:
            return detections

        result = results[0]
        has_masks = result.masks is not None and len(result.masks) > 0

        for i, box in enumerate(result.boxes):
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = float(box.conf[0])
            cls_id = int(box.cls[0])
            label = self.model.names[cls_id]

            bx, by = max(0, x1), max(0, y1)
            bw = min(x2, w) - bx
            bh = min(y2, h) - by

            seg_mask = None
            mask_area = 0

            if has_masks:
                raw_mask = result.masks.data[i].cpu().numpy()
                seg_mask = cv2.resize(raw_mask, (w, h), interpolation=cv2.INTER_LINEAR)
                seg_mask = (seg_mask > 0.5).astype(np.uint8) * 255
                mask_area = int(np.sum(seg_mask > 0))
            else:
                seg_mask = np.zeros((h, w), dtype=np.uint8)
                seg_mask[by:by + bh, bx:bx + bw] = 255
                mask_area = bw * bh

            depths = []
            mask_coords = np.where(seg_mask > 0)
            step = max(1, len(mask_coords[0]) // 500)
            for idx in range(0, len(mask_coords[0]), step):
                py, px = mask_coords[0][idx], mask_coords[1][idx]
                d = depth_frame.get_distance(int(px), int(py))
                if self.min_depth < d < self.max_depth:
                    depths.append(d)

            if depths:
                avg_depth = float(np.mean(depths))
                min_depth = float(np.min(depths))
                max_depth = float(np.max(depths))
            else:
                avg_depth = min_depth = max_depth = 0.0

            detections.append({
                'bbox': (bx, by, bw, bh),
                'conf': conf,
                'cls_id': cls_id,
                'label': label,
                'seg_mask': seg_mask,
                'mask_area': mask_area,
                'avg_depth': avg_depth,
                'min_depth': min_depth,
                'max_depth': max_depth,
            })

        detections.sort(key=lambda d: d['conf'], reverse=True)
        return detections

    def compute_real_dimensions(self, det, depth_frame, color_frame):
        intrinsics = color_frame.profile.as_video_stream_profile().intrinsics
        x, y, bw, bh = det['bbox']

        corners_2d = [(x, y), (x + bw, y), (x, y + bh), (x + bw, y + bh)]
        corners_3d = []
        for (cx, cy) in corners_2d:
            cx = min(max(cx, 0), intrinsics.width - 1)
            cy = min(max(cy, 0), intrinsics.height - 1)
            d = depth_frame.get_distance(cx, cy)
            if d <= 0 or d > self.max_depth:
                d = det['avg_depth']
            if d > 0:
                pt = rs.rs2_deproject_pixel_to_point(intrinsics, [cx, cy], d)
                corners_3d.append(np.array(pt))

        if len(corners_3d) == 4:
            w_mm = np.linalg.norm(corners_3d[1] - corners_3d[0]) * 1000
            h_mm = np.linalg.norm(corners_3d[2] - corners_3d[0]) * 1000
            t_mm = (det['max_depth'] - det['min_depth']) * 1000
            return w_mm, h_mm, t_mm
        return None, None, None

    def export_object_3d(self, depth_frame, color_frame, det):
        """
        Xuất 3D mesh chính xác theo hình dáng vật thể.
        Dùng seg mask + Poisson reconstruction.
        """
        try:
            print("\n" + "=" * 55)
            print(f"[3D EXPORT] Đang tạo mesh cho: {det['label']}...")
            print(f"            Mask pixels: {det['mask_area']}")
            print(f"            Depth range: {det['min_depth']:.3f}m - {det['max_depth']:.3f}m")

            color_image = np.asanyarray(color_frame.get_data())
            h, w = color_image.shape[:2]
            color_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

            seg_mask = det['seg_mask']
            if seg_mask is None or np.sum(seg_mask > 0) < 1000:
                print("[LỖI] Mask quá nhỏ hoặc không có.")
                return

            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
            seg_mask = cv2.morphologyEx(seg_mask, cv2.MORPH_CLOSE, kernel)
            seg_mask = cv2.morphologyEx(seg_mask, cv2.MORPH_OPEN, kernel)

            pc = rs.pointcloud()
            pc.map_to(color_frame)
            points_3d = pc.calculate(depth_frame)
            v = points_3d.get_vertices()
            verts = np.asanyarray(v).view(np.float32).reshape(h, w, 3)

            d_margin = 0.03
            d_min = max(self.min_depth, det['min_depth'] - d_margin)
            d_max = min(self.max_depth, det['max_depth'] + d_margin)

            valid = (
                (seg_mask > 0) &
                (verts[:, :, 2] > d_min) &
                (verts[:, :, 2] < d_max) &
                (verts[:, :, 0] != 0) &
                (verts[:, :, 1] != 0)
            )

            valid_indices = np.where(valid)
            final_verts = verts[valid_indices]
            final_colors = color_rgb[valid_indices]

            print(f"            Valid 3D points: {len(final_verts)}")
            if len(final_verts) < 2000:
                print("[LỖI] Không đủ điểm 3D. Đưa vật thể gần hơn (0.3-0.8m).")
                return

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(final_verts)
            pcd.colors = o3d.utility.Vector3dVector(final_colors)

            pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=30, std_ratio=1.5)
            print(f"            After outlier removal: {len(pcd.points)} points")

            pcd.translate(-pcd.get_center())
            R = pcd.get_rotation_matrix_from_axis_angle([np.pi, 0, 0])
            pcd.rotate(R, center=(0, 0, 0))

            pcd.estimate_normals(
                search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.02, max_nn=50)
            )
            pcd.orient_normals_towards_camera_location(camera_location=[0, 0, -1])

            print(f"            Poisson reconstruction (depth={self.poisson_depth})...")
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=self.poisson_depth, width=0, scale=1.1, linear_fit=False
            )

            densities = np.asarray(densities)
            density_threshold = np.quantile(densities, 0.05)
            vertices_to_remove = densities < density_threshold
            mesh.remove_vertices_by_mask(vertices_to_remove)

            bbox = pcd.get_axis_aligned_bounding_box()
            bbox_min = bbox.get_min_bound() - np.abs(bbox.get_min_bound()) * 0.05
            bbox_max = bbox.get_max_bound() + np.abs(bbox.get_max_bound()) * 0.05
            crop_bbox = o3d.geometry.AxisAlignedBoundingBox(bbox_min, bbox_max)
            mesh = mesh.crop(crop_bbox)

            # Map vertex colors từ point cloud
            mesh.paint_uniform_color([0.7, 0.7, 0.7])
            mesh_verts = np.asarray(mesh.vertices)
            pcd_tree = o3d.geometry.KDTreeFlann(pcd)
            vertex_colors = np.zeros_like(mesh_verts)
            pcd_colors = np.asarray(pcd.colors)
            for i in range(len(mesh_verts)):
                _, idx, _ = pcd_tree.search_knn_vector_3d(mesh_verts[i], 1)
                vertex_colors[i] = pcd_colors[idx[0]]
            mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)

            mesh = mesh.filter_smooth_laplacian(number_of_iterations=3)
            mesh.compute_vertex_normals()

            safe_label = det['label'].replace(' ', '_')
            filename = f"object_{safe_label}_{det['avg_depth']:.1f}m.ply"
            o3d.io.write_triangle_mesh(filename, mesh)
            full_path = os.path.abspath(filename)

            print(f"\n[THÀNH CÔNG] File: {full_path}")
            print(f"             Label: {det['label']} ({det['conf']:.0%})")
            print(f"             Vertices: {len(mesh.vertices)}")
            print(f"             Triangles: {len(mesh.triangles)}")

            os.startfile(full_path)

        except Exception as e:
            import traceback
            print(f"[LỖI] {e}")
            traceback.print_exc()

    def run(self):
        pipeline_started = False
        try:
            profile = self.pipeline.start(self.config)
            pipeline_started = True

            device = profile.get_device()
            depth_sensor = device.query_sensors()[0]
            if depth_sensor.supports(rs.option.visual_preset):
                depth_sensor.set_option(rs.option.visual_preset, 4)

            window_name = 'YOLOv8-Seg + RealSense 3D Scanner'
            cv2.namedWindow(window_name)
            cv2.setMouseCallback(window_name, self.mouse_callback)

            print("\n" + "=" * 55)
            print("  YOLOv8-Seg + RealSense D455 — 3D Object Scanner")
            print("=" * 55)
            print("  Click chuột  → Chọn vật thể")
            print("  [s]          → Xuất 3D mesh (.ply)")
            print("  [m]          → Bật/tắt lưới mesh wireframe")
            print("  [+/-]        → Tăng/giảm mật độ lưới")
            print("  [c]          → Xoá lựa chọn")
            print("  [q]          → Thoát")
            print("=" * 55)

            while True:
                frames = self.pipeline.wait_for_frames()
                aligned = self.align.process(frames)
                depth_frame = aligned.get_depth_frame()
                color_frame = aligned.get_color_frame()
                if not depth_frame or not color_frame:
                    continue

                depth_frame = self.spatial.process(depth_frame)
                depth_frame = self.temporal.process(depth_frame)
                depth_frame = self.hole_filling.process(depth_frame).as_depth_frame()

                color_image = np.asanyarray(color_frame.get_data())
                h, w = color_image.shape[:2]

                # ===== YOLO-SEG DETECT =====
                detections = self.detect_objects(color_image, depth_frame, h, w)
                self._current_detections = detections

                # ===== DASHBOARD =====
                overlay = color_image.copy()
                panel_h = 55 + len(detections[:8]) * 22
                cv2.rectangle(overlay, (5, 5), (370, max(65, panel_h)), (0, 0, 0), -1)
                cv2.addWeighted(overlay, 0.65, color_image, 0.35, 0, color_image)

                d_center = self.get_average_depth(depth_frame, w // 2, h // 2, h, w)
                clr = (0, 255, 0) if 0.3 <= d_center <= 1.5 else (0, 0, 255)
                cv2.putText(color_image, f"Distance: {d_center:.2f}m", (15, 28), 0, 0.6, clr, 2)

                mesh_status = "MESH ON" if self.show_mesh_wireframe else "MESH OFF"
                cv2.putText(color_image, f"Objects: {len(detections)} | {mesh_status} (grid={self.mesh_grid_step}px)",
                            (15, 50), 0, 0.42, (0, 255, 255), 1)

                # ===== VẼ LƯỚI MESH WIREFRAME + MASKS =====
                for i, det in enumerate(detections):
                    cls_id = det['cls_id']
                    color = self.class_colors.get(cls_id, (0, 255, 0))
                    bx, by, bw, bh = det['bbox']

                    is_selected = (
                        self.selected_object is not None and
                        det['bbox'] == self.selected_object['bbox']
                    )

                    # ★ VẼ LƯỚI MESH WIREFRAME (giống Face Mesh tesselation) ★
                    if self.show_mesh_wireframe and det['avg_depth'] > 0:
                        self.draw_mesh_wireframe(color_image, det, depth_frame, h, w)

                    # Bounding box
                    thickness = 3 if is_selected else 1
                    cv2.rectangle(color_image, (bx, by), (bx + bw, by + bh), color, thickness)

                    if is_selected:
                        cv2.rectangle(color_image, (bx - 2, by - 2),
                                      (bx + bw + 2, by + bh + 2), (255, 255, 255), 1)

                    # Label
                    label_text = f"{det['label']} {det['conf']:.0%}"
                    if det['avg_depth'] > 0:
                        label_text += f" {det['avg_depth']:.2f}m"

                    (tw, th_t), _ = cv2.getTextSize(label_text, 0, 0.5, 1)
                    cv2.rectangle(color_image, (bx, by - th_t - 8),
                                  (bx + tw + 4, by), color, -1)
                    cv2.putText(color_image, label_text, (bx + 2, by - 5),
                                0, 0.5, (255, 255, 255), 1)

                    # Kích thước thực
                    rw, rh_mm, t_mm = self.compute_real_dimensions(det, depth_frame, color_frame)
                    if rw is not None:
                        dim_text = f"{rw:.0f}x{rh_mm:.0f}mm"
                        if t_mm and t_mm > 3:
                            dim_text += f" D:{t_mm:.0f}mm"
                        cv2.putText(color_image, dim_text,
                                    (bx, by + bh + 16), 0, 0.45, color, 1)

                    # Dashboard
                    if i < 8:
                        info_y = 70 + i * 22
                        prefix = ">>>" if is_selected else f" {i + 1}."
                        info = f"{prefix} {det['label']} ({det['conf']:.0%}) d={det['avg_depth']:.2f}m"
                        txt_clr = (255, 255, 255) if is_selected else (200, 200, 200)
                        cv2.putText(color_image, info, (15, info_y), 0, 0.4, txt_clr, 1)

                # Click marker
                if self.clicked_point:
                    cx, cy = self.clicked_point
                    d_click = self.get_average_depth(depth_frame, cx, cy, h, w)
                    cv2.drawMarker(color_image, (cx, cy), (0, 0, 255),
                                   cv2.MARKER_CROSS, 20, 2)
                    if d_click > 0:
                        cv2.putText(color_image, f"{d_click:.3f}m",
                                    (cx + 12, cy - 8), 0, 0.55, (0, 255, 255), 2)

                cv2.imshow(window_name, color_image)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('s'):
                    target = self.selected_object if self.selected_object else (
                        detections[0] if detections else None
                    )
                    if target:
                        self.export_object_3d(depth_frame, color_frame, target)
                    else:
                        print("[INFO] Không có vật thể nào để xuất.")
                elif key == ord('m'):
                    self.show_mesh_wireframe = not self.show_mesh_wireframe
                    print(f"[MESH] Wireframe: {'ON' if self.show_mesh_wireframe else 'OFF'}")
                elif key == ord('+') or key == ord('='):
                    self.mesh_grid_step = max(3, self.mesh_grid_step - 2)
                    print(f"[MESH] Grid step: {self.mesh_grid_step}px (dày hơn)")
                elif key == ord('-'):
                    self.mesh_grid_step = min(30, self.mesh_grid_step + 2)
                    print(f"[MESH] Grid step: {self.mesh_grid_step}px (thưa hơn)")
                elif key == ord('c'):
                    self.clicked_point = None
                    self.selected_object = None

        except Exception as e:
            print(f"Lỗi: {e}")
        finally:
            if pipeline_started:
                self.pipeline.stop()
            cv2.destroyAllWindows()


if __name__ == "__main__":
    # yolov8n-seg.pt = nano   (~7MB,  nhanh, có seg mask)
    # yolov8s-seg.pt = small  (~23MB, cân bằng)
    # yolov8m-seg.pt = medium (~52MB, chính xác hơn)
    # yolov8x-seg.pt = xlarge (~137MB, chính xác nhất, cần GPU)
    scanner = RealSenseYOLOScanner(model_name="yolov8n-seg.pt", confidence=0.45)
    scanner.run()

[YOLO-SEG] Đang tải model...
[YOLO-SEG] Model 'yolov8n-seg.pt' đã sẵn sàng!

  YOLOv8-Seg + RealSense D455 — 3D Object Scanner
  Click chuột  → Chọn vật thể
  [s]          → Xuất 3D mesh (.ply)
  [m]          → Bật/tắt lưới mesh wireframe
  [+/-]        → Tăng/giảm mật độ lưới
  [c]          → Xoá lựa chọn
  [q]          → Thoát


In [20]:
import torch
from torchinfo import summary
# In ra type của chuỗi, thông tin file model và kiến trúc mô hình đã load (dùng scanner từ cell trước)
model_path = 'yolov8n-seg.pt'
print("type('yolov8n-seg.pt'):", type(model_path))

print("Tồn tại:", os.path.exists(model_path))
if os.path.exists(model_path):
    print("Kích thước (bytes):", os.path.getsize(model_path))
    print("Đường dẫn đầy đủ:", os.path.abspath(model_path))

print("\nscanner.model (YOLO object) type:", type(scanner.model))
try:
    torch_model = scanner.model.model  # underlying torch.nn.Module
    print("Underlying torch model type:", type(torch_model))
    # In mô tả ngắn và số lượng tham số
    print("\n--- Tóm tắt mô hình (repr) ---")
    print(torch_model)
    try:
        print("\n--- Chi tiết kiến trúc từ torchinfo.summary ---")
        summary(torch_model, input_size=(1, 3, 640, 640))
    except Exception as e:
        print("Không thể chạy torchinfo.summary:", e)
    try:
        total_params = sum(p.numel() for p in torch_model.parameters())
        trainable = sum(p.numel() for p in torch_model.parameters() if p.requires_grad)
        print(f"\nTổng tham số: {total_params:,}, Trainable: {trainable:,}")
    except Exception:
        pass
except Exception as e:
    print("Không thể truy cập scanner.model.model:", e)


type('yolov8n-seg.pt'): <class 'str'>
Tồn tại: True
Kích thước (bytes): 7071756
Đường dẫn đầy đủ: d:\FaceGrid3D\yolov8n-seg.pt

scanner.model (YOLO object) type: <class 'ultralytics.models.yolo.model.YOLO'>
Underlying torch model type: <class 'ultralytics.nn.tasks.SegmentationModel'>

--- Tóm tắt mô hình (repr) ---
SegmentationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): ModuleList(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(16, 16, ke